# ccxt를 이용한 데이터 수집

In [ ]:
pip install ccxt

In [1]:
# 라이브러리 import
import ccxt
import pandas as pd

In [2]:
# binance 변수 설정
binance = ccxt.binance()

In [4]:
# 분 단위(default)
## BTC 데이터 불러오기
btc_ohlcv = binance.fetch_ohlcv("BTC/USDT")
## 데이터프레임 생성
btc_ohlcv_df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
btc_ohlcv_df['datetime'] = pd.to_datetime(btc_ohlcv_df['datetime'], unit='ms')
btc_ohlcv_df.set_index('datetime', inplace=True)
## 데이터프레임 출력
print(btc_ohlcv_df)

                         open      high       low     close    volume
datetime                                                             
2022-01-20 07:52:00  42038.41  42043.07  42018.81  42043.07   8.29377
2022-01-20 07:53:00  42043.06  42071.54  42043.06  42049.21  15.89499
2022-01-20 07:54:00  42049.21  42064.19  42037.07  42041.46   9.50859
2022-01-20 07:55:00  42041.46  42054.21  42018.80  42040.75  13.32876
2022-01-20 07:56:00  42040.75  42041.74  42027.34  42027.35   7.31216
...                       ...       ...       ...       ...       ...
2022-01-20 16:07:00  43386.67  43389.74  43330.01  43361.88  45.62718
2022-01-20 16:08:00  43358.32  43363.27  43318.90  43322.01  37.98414
2022-01-20 16:09:00  43322.01  43327.29  43283.80  43297.69  30.91202
2022-01-20 16:10:00  43297.69  43297.70  43223.16  43233.97  40.11309
2022-01-20 16:11:00  43233.98  43258.79  43200.35  43212.04  27.33035

[500 rows x 5 columns]


In [20]:
# 시간 단위
## BTC 데이터 불러오기
## '1d' 변수 인자 추가
btc_ohlcv = binance.fetch_ohlcv("BTC/USDT", '1h')
## 데이터프레임 생성
btc_ohlcv_df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
btc_ohlcv_df['datetime'] = pd.to_datetime(btc_ohlcv_df['datetime'], unit='ms')
btc_ohlcv_df.set_index('datetime', inplace=True)
## 데이터프레임 출력
print(btc_ohlcv_df)

                         open      high       low     close      volume
datetime                                                               
2021-12-30 21:00:00  47203.16  47347.84  46919.44  47300.00   890.70108
2021-12-30 22:00:00  47299.99  47325.33  46729.00  47084.88   792.93614
2021-12-30 23:00:00  47084.88  47242.60  46940.96  47120.87   710.18694
2021-12-31 00:00:00  47120.88  47500.00  46826.21  47073.98  1084.11426
2021-12-31 01:00:00  47073.99  47349.10  46842.28  47121.94   762.07203
...                       ...       ...       ...       ...         ...
2022-01-20 12:00:00  42129.45  42131.71  41900.05  42033.76  1141.58213
2022-01-20 13:00:00  42033.77  42466.88  41970.19  42375.14  1980.38037
2022-01-20 14:00:00  42374.09  43060.00  42368.77  42969.66  3692.57219
2022-01-20 15:00:00  42969.67  43456.00  42936.91  43308.69  4058.16745
2022-01-20 16:00:00  43308.70  43505.00  43144.84  43208.56  1210.07187

[500 rows x 5 columns]


* 시간 단위 : '1h' 변수 추가
* 일 단위 : '1d' 변수 추가
* 주 단위 : '1w' 변수 추가
* 월 단위 : '1M' 변수 추가
* 월 단위 : '1M' 변수 추가

# python-binance를 이용한 데이터 수집

In [ ]:
pip install python-binance

# mySQL을 이용한 데이터 수집

In [ ]:
pip install pymysql

In [1]:
# 라이브러리 import
import ccxt
import requests
import pandas as pd
import datetime
import time
import pymysql

In [2]:
# 기본 변수 설정
n = 0
binance = ccxt.binance()

# mysql 계정 정보 입력
##### password 변경 필요
con = pymysql.connect(host="localhost", user="root", password="password", db="coinanalysis", charset="utf8")
cur = con.cursor()

In [6]:
# binance_historical_data 테이블에 데이터 저장
# binance의 모든 market에 대해 2021-12-22 09:00:00 부터 일 단위 과거 데이터 수집

var_final_date = '2021-12-22 09:00:00'          # 조회를 시작하고자 하는 날짜 입력

all_tickers = binance.load_markets()
tickers = []          # 코인별 종목코드 - 모든 종목에 대한 데이터 수집이 필요할 경우

for ticker in all_tickers.keys():
    if ticker.endswith("/USDT"):
        ticker = str(ticker)
        ticker = ticker.replace("/", "")
        tickers.append(ticker)

for ticker in tickers:

    n = n + 1
    loop_check = True

    target_date = datetime.datetime.strptime(var_final_date, '%Y-%m-%d %H:%M:%S')
    target_date_timetuple = target_date.timetuple()
    target_date_timestamp = time.mktime(target_date_timetuple)
    target_date_timestamp = int(target_date_timestamp * 1000)

    while loop_check is True:

        print(target_date)

        url = "https://api.binance.com/api/v3/klines?symbol=" + ticker + "&interval=1d&endTime=" + str(target_date_timestamp) + "&limit=1000"
          # interval로 단위 변경 - 현재 일 단위
        resp = requests.get(url)
        data = resp.json()

        col_name = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trade_num',
                    'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore']

        df = pd.DataFrame(data, columns=col_name)

        df = df.drop(['close_time', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'], axis='columns')
        df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'trade_num']]

        if len(df) == 0:
            break

        for i in range(len(df)):
            date_time_timestamp = df.iat[i, 0]
            date_time = datetime.datetime.fromtimestamp(date_time_timestamp / 1000)
            if i == 0:
                temp_date = date_time
            open = float(df.iat[i, 1])
            high = float(df.iat[i, 2])
            low = float(df.iat[i, 3])
            close = float(df.iat[i, 4])
            volume = float(df.iat[i, 5])
            value = float(df.iat[i, 6])
            trade_num = float(df.iat[i, 7])

            sql = "insert into binance_historical_data values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.execute(sql, (ticker, date_time, open, high, low, close, volume, value, trade_num))

        if len(df) < 1000:
            break

        target_date = temp_date - datetime.timedelta(1)
        target_date_timetuple = target_date.timetuple()
        target_date_timestamp = time.mktime(target_date_timetuple)
        target_date_timestamp = int(target_date_timestamp * 1000)

        time.sleep(0.05)

    print(str(n) + ". " + ticker)

con.commit()

2021-12-22 09:00:00


IntegrityError: (1062, "Duplicate entry 'BTCUSDT-2017-08-21 09:00:00' for key 'binance_historical_data.PRIMARY'")

In [8]:
# test_data_table_1 테이블에 데이터 저장
# ETH에 대해 2021-12-22 09:00:00 부터 시간 단위 과거 데이터 수집

var_final_date = '2021-12-22 09:00:00'          # 조회를 시작하고자 하는 날짜 입력

tickers = ["ETHUSDT"]          # 코인별 종목코드 - ETH/USDT 하나만 선택한 경우

for ticker in tickers:

    n = n + 1
    loop_check = True

    target_date = datetime.datetime.strptime(var_final_date, '%Y-%m-%d %H:%M:%S')
    target_date_timetuple = target_date.timetuple()
    target_date_timestamp = time.mktime(target_date_timetuple)
    target_date_timestamp = int(target_date_timestamp * 1000)

    while loop_check is True:

        print(target_date)

        url = "https://api.binance.com/api/v3/klines?symbol=" + ticker + "&interval=1h&endTime=" + str(target_date_timestamp) + "&limit=1000"
          # interval로 단위 변경 - 현재 시간 단위
        resp = requests.get(url)
        data = resp.json()

        col_name = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trade_num',
                    'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore']

        df = pd.DataFrame(data, columns=col_name)

        df = df.drop(['close_time', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'], axis='columns')
        df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'trade_num']]

        if len(df) == 0:
            break

        for i in range(len(df)):
            date_time_timestamp = df.iat[i, 0]
            date_time = datetime.datetime.fromtimestamp(date_time_timestamp / 1000)
            if i == 0:
                temp_date = date_time
            open = float(df.iat[i, 1])
            high = float(df.iat[i, 2])
            low = float(df.iat[i, 3])
            close = float(df.iat[i, 4])
            volume = float(df.iat[i, 5])
            value = float(df.iat[i, 6])
            trade_num = float(df.iat[i, 7])

            sql = "insert into test_data_table_1 values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.execute(sql, (ticker, date_time, open, high, low, close, volume, value, trade_num))

        if len(df) < 1000:
            break

        target_date = temp_date - datetime.timedelta(1)
        target_date_timetuple = target_date.timetuple()
        target_date_timestamp = time.mktime(target_date_timetuple)
        target_date_timestamp = int(target_date_timestamp * 1000)

        time.sleep(0.05)

    print(str(n) + ". " + ticker)

con.commit()

2021-12-22 09:00:00
2021-11-09 18:00:00
2021-09-28 01:00:00
2021-08-16 10:00:00
2021-07-04 15:00:00
2021-05-23 00:00:00
2021-04-10 04:00:00
2021-02-26 12:00:00
2021-01-14 20:00:00
2020-12-03 01:00:00
2020-10-21 09:00:00
2020-09-08 18:00:00
2020-07-28 03:00:00
2020-06-15 09:00:00
2020-05-03 18:00:00
2020-03-22 01:00:00
2020-02-08 03:00:00
2019-12-27 12:00:00
2019-11-14 19:00:00
2019-10-03 02:00:00
2019-08-21 11:00:00
2019-07-09 12:00:00
2019-05-27 21:00:00
2019-04-14 20:00:00
2019-03-02 23:00:00
2019-01-19 08:00:00
2018-12-07 17:00:00
2018-10-25 19:00:00
2018-09-13 01:00:00
2018-08-01 10:00:00
2018-06-19 01:00:00
2018-05-07 10:00:00
2018-03-25 19:00:00
2018-02-11 04:00:00
2017-12-29 03:00:00
2017-11-16 12:00:00
2017-10-04 21:00:00
2017-08-22 23:00:00
1. ETHUSDT


In [5]:
# BTC_220127_1d 테이블에 데이터 저장
# BTC에 대해 2022-01-27 09:00:00 부터 일 단위 과거 데이터 수집

var_final_date = '2022-01-27 09:00:00'          # 조회를 시작하고자 하는 날짜 입력

tickers = ["BTCUSDT"]          # 코인별 종목코드 - BTC/USDT 하나만 선택한 경우

for ticker in tickers:

    n = n + 1
    loop_check = True

    target_date = datetime.datetime.strptime(var_final_date, '%Y-%m-%d %H:%M:%S')
    target_date_timetuple = target_date.timetuple()
    target_date_timestamp = time.mktime(target_date_timetuple)
    target_date_timestamp = int(target_date_timestamp * 1000)

    while loop_check is True:

        print(target_date)

        url = "https://api.binance.com/api/v3/klines?symbol=" + ticker + "&interval=1d&endTime=" + str(target_date_timestamp) + "&limit=1000"
          # interval로 단위 변경 - 현재 시간 단위
        resp = requests.get(url)
        data = resp.json()

        col_name = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trade_num',
                    'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore']

        df = pd.DataFrame(data, columns=col_name)

        df = df.drop(['close_time', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'], axis='columns')
        df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'trade_num']]

        if len(df) == 0:
            break

        for i in range(len(df)):
            date_time_timestamp = df.iat[i, 0]
            date_time = datetime.datetime.fromtimestamp(date_time_timestamp / 1000)
            if i == 0:
                temp_date = date_time
            open = float(df.iat[i, 1])
            high = float(df.iat[i, 2])
            low = float(df.iat[i, 3])
            close = float(df.iat[i, 4])
            volume = float(df.iat[i, 5])
            value = float(df.iat[i, 6])
            trade_num = float(df.iat[i, 7])

            sql = "insert into BTC_220127_1d values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.execute(sql, (ticker, date_time, open, high, low, close, volume, value, trade_num))

        if len(df) < 1000:
            break

        target_date = temp_date - datetime.timedelta(1)
        target_date_timetuple = target_date.timetuple()
        target_date_timestamp = time.mktime(target_date_timetuple)
        target_date_timestamp = int(target_date_timestamp * 1000)

        time.sleep(0.05)

    print(str(n) + ". " + ticker)

con.commit()

2022-01-27 09:00:00
2019-05-03 09:00:00
3. BTCUSDT


In [6]:
# ETH_220127_1d 테이블에 데이터 저장
# ETH에 대해 2022-01-27 09:00:00 부터 일 단위 과거 데이터 수집

var_final_date = '2022-01-27 09:00:00'          # 조회를 시작하고자 하는 날짜 입력

tickers = ["ETHUSDT"]          # 코인별 종목코드 - ETH/USDT 하나만 선택한 경우

for ticker in tickers:

    n = n + 1
    loop_check = True

    target_date = datetime.datetime.strptime(var_final_date, '%Y-%m-%d %H:%M:%S')
    target_date_timetuple = target_date.timetuple()
    target_date_timestamp = time.mktime(target_date_timetuple)
    target_date_timestamp = int(target_date_timestamp * 1000)

    while loop_check is True:

        print(target_date)

        url = "https://api.binance.com/api/v3/klines?symbol=" + ticker + "&interval=1d&endTime=" + str(target_date_timestamp) + "&limit=1000"
          # interval로 단위 변경 - 현재 시간 단위
        resp = requests.get(url)
        data = resp.json()

        col_name = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trade_num',
                    'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore']

        df = pd.DataFrame(data, columns=col_name)

        df = df.drop(['close_time', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'], axis='columns')
        df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'trade_num']]

        if len(df) == 0:
            break

        for i in range(len(df)):
            date_time_timestamp = df.iat[i, 0]
            date_time = datetime.datetime.fromtimestamp(date_time_timestamp / 1000)
            if i == 0:
                temp_date = date_time
            open = float(df.iat[i, 1])
            high = float(df.iat[i, 2])
            low = float(df.iat[i, 3])
            close = float(df.iat[i, 4])
            volume = float(df.iat[i, 5])
            value = float(df.iat[i, 6])
            trade_num = float(df.iat[i, 7])

            sql = "insert into ETH_220127_1d values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.execute(sql, (ticker, date_time, open, high, low, close, volume, value, trade_num))

        if len(df) < 1000:
            break

        target_date = temp_date - datetime.timedelta(1)
        target_date_timetuple = target_date.timetuple()
        target_date_timestamp = time.mktime(target_date_timetuple)
        target_date_timestamp = int(target_date_timestamp * 1000)

        time.sleep(0.05)

    print(str(n) + ". " + ticker)

con.commit()

2022-01-27 09:00:00
2019-05-03 09:00:00
4. ETHUSDT
